In [26]:
import numpy as np
import pandas as pd
import re


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings(action='ignore')

# TO USE XGBOOST YOU NEED TO RUN ' conda install -c anaconda py-xgboost ' IN TERMINAL THEN RESTART JUPYTER

In [27]:
# get data

train = pd.read_csv('df - train_set.csv')
test = pd.read_csv('df - test_set.csv')

train = train[(train['Commodities'] == 'APPLE GOLDEN DELICIOUS')]
del train['Commodities'] 

train.head()

,Province,Container,Size_Grade,Weight_Kg,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,avg_price_per_kg
1,CAPE,M4183,1L,18.3,2020-09-09,150.0,170.0,51710.0,332,6075.6,822,8.51
7,CAPE,JG110,2M,11.0,2020-04-14,50.0,50.0,16000.0,320,3520.0,0,4.55
24,W.CAPE-BERGRIVER ETC,JE090,2S,9.0,2020-04-16,55.0,55.0,990.0,18,162.0,1506,6.11
40,CAPE,M4183,1S,18.3,2020-05-04,80.0,120.0,32020.0,388,7100.4,443,4.51
69,EASTERN CAPE,IA400,1S,400.0,2020-09-28,1800.0,1800.0,1800.0,1,400.0,2,4.50


In [35]:
del test['Commodities']
test.head()

,Index,Province,Container,Size_Grade,Weight_Kg,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand
0,1,W.CAPE-BERGRIVER ETC,EC120,1M,12.0,2020-07-09,128.0,136.0,5008.0,38,456.0,0
1,2,W.CAPE-BERGRIVER ETC,M4183,1X,18.3,2020-01-20,220.0,220.0,1760.0,8,146.4,2
2,3,W.CAPE-BERGRIVER ETC,EC120,1S,12.0,2020-08-19,120.0,120.0,720.0,6,72.0,45
3,4,W.CAPE-BERGRIVER ETC,M4183,1M,18.3,2020-05-06,160.0,160.0,160.0,1,18.3,8
4,5,W.CAPE-BERGRIVER ETC,M4183,1L,18.3,2020-05-04,140.0,160.0,14140.0,100,1830.0,19


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 685 entries, 0 to 684
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Index           685 non-null    int64  
 1   Province        685 non-null    object 
 2   Container       685 non-null    object 
 3   Size_Grade      685 non-null    object 
 4   Weight_Kg       685 non-null    float64
 5   Date            685 non-null    object 
 6   Low_Price       685 non-null    float64
 7   High_Price      685 non-null    float64
 8   Sales_Total     685 non-null    float64
 9   Total_Qty_Sold  685 non-null    int64  
 10  Total_Kg_Sold   685 non-null    float64
 11  Stock_On_Hand   685 non-null    int64  
dtypes: float64(5), int64(3), object(4)
memory usage: 64.3+ KB


In [6]:
train.describe()

,Weight_Kg,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,avg_price_per_kg
count,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000
mean,40.460912,174.307377,215.648053,20053.533811,174.510758,2960.176332,408.393955,6.778893
std,99.655169,373.553578,433.546159,39005.069445,308.810797,6097.416527,724.450582,2.248744
min,3.000000,2.000000,5.000000,5.000000,1.000000,3.000000,0.000000,0.250000
25%,9.000000,50.000000,60.000000,1325.000000,12.000000,219.600000,9.000000,5.460000
50%,12.000000,80.000000,108.000000,5495.000000,64.000000,853.500000,126.500000,6.670000
75%,18.300000,127.250000,160.000000,21082.500000,200.000000,3093.525000,468.000000,8.280000
max,400.000000,2300.000000,3300.000000,369464.000000,4237.000000,74000.000000,6400.000000,21.240000


In [264]:

train

,Province,Container,Size_Grade,Weight_Kg,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,avg_price_per_kg
1,CAPE,M4183,1L,18.3,2020-09-09,150.0,170.0,51710.0,332,6075.6,822,8.51
7,CAPE,JG110,2M,11.0,2020-04-14,50.0,50.0,16000.0,320,3520.0,0,4.55
24,W.CAPE-BERGRIVER ETC,JE090,2S,9.0,2020-04-16,55.0,55.0,990.0,18,162.0,1506,6.11
40,CAPE,M4183,1S,18.3,2020-05-04,80.0,120.0,32020.0,388,7100.4,443,4.51
69,EASTERN CAPE,IA400,1S,400.0,2020-09-28,1800.0,1800.0,1800.0,1,400.0,2,4.50
...,...,...,...,...,...,...,...,...,...,...,...,...
64276,NATAL,EC120,1S,12.0,2020-03-04,96.0,96.0,1920.0,20,240.0,53,8.00
64291,ORANGE FREE STATE,M4183,1X,18.3,2020-02-22,130.0,130.0,7280.0,56,1024.8,74,7.10
64297,CAPE,JE090,2M,9.0,2020-04-22,50.0,60.0,4540.0,89,801.0,793,5.67
64304,CAPE,JG110,2M,11.0,2020-03-04,30.0,40.0,2140.0,70,770.0,0,2.78


In [7]:
train['Province'].unique()

array(['CAPE', 'W.CAPE-BERGRIVER ETC', 'EASTERN CAPE', 'NATAL',
       'WEST COAST', 'TRANSVAAL', 'ORANGE FREE STATE'], dtype=object)

In [8]:
train['Container'].unique()

array(['M4183', 'JG110', 'JE090', 'IA400', 'EC120', 'AC030', 'M6125',
       'EF120', 'DT063', 'M9125', 'EG140'], dtype=object)

In [9]:
train['Size_Grade'].unique()

array(['1L', '2M', '2S', '1S', '1M', '1X', '2L', '2U', '2X', '1U'],
      dtype=object)

In [28]:
# DUMMY

def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column, drop_first=True)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [107]:
# PREPROCESSING 

def preprocess_inputs(df):
    df = df.copy()
    
    # CLEAN PROVINCE COLUMN
    
    df['Province'] = df['Province'].str.replace(' ', '_')
    df['Province'] = df['Province'].str.replace('.', '_')
    df['Province'] = df['Province'].str.replace('-', '_')
    
    # DATE ENCODING
    # Split 'Date' column into year, month and day columns 

    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].dt.strftime('%d.%m.%Y')
    df['year'] = pd.DatetimeIndex(df['Date']).year
    df['month'] = pd.DatetimeIndex(df['Date']).month
    df['day'] = pd.DatetimeIndex(df['Date']).day

    df = df.drop(['Date'], axis = 1) 
    
    # BINARY ENCODING
    
    df['year'] = df['year'].replace({2020: 1, 2019: 0})
        
    # ONE-HOT ENCODING
    for column in ['Province', 'Container', 'Size_Grade']:
        df = onehot_encode(df, column)
    
    # MAYBE REMOVE REMORE HIGHLY CORRRELATED COLUMNS BY UNCOMMENTING 2 LINES BELOW, I HAVEN'T
    
    #for column in ['Total_Kg_Sold', 'Sales_Total', 'Low_Price']:
    #    df = df.drop([column], axis = 1) 
        
    
        ## for training dataset
        # REORDER COLUMNS SO THAT OUR DEPENDENT VARIABLE IS THE LAST COLUMN OF THE DATAFRAME
    if 'avg_price_per_kg' in df.columns:
        column_titles = [col for col in df.columns if col!= 'avg_price_per_kg'] + ['avg_price_per_kg']
        df = df.reindex(columns = column_titles)
    
        # SPLIT DATA INTO PREDICTORS AND TARGET
    
        y = df['avg_price_per_kg']
        X = df.drop('avg_price_per_kg', axis=1)

        # TRAIN TEST SPLIT
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1)

        # MAYBE SCALE,  
        
        #scaler = StandardScaler()
        #scaler.fit(X_train)
        #X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
        #X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
        
        return X_train, X_test, y_train, y_test
    
    else:
        return df
    

In [108]:
X_train, X_test, y_train, y_test = preprocess_inputs(train)

In [109]:
# TRAIN A FEW MODELS

models = {
    "                     Linear Regression": LinearRegression(),
    " Linear Regression (L2 Regularization)": Ridge(),
    " Linear Regression (L1 Regularization)": Lasso(),
    "                   K-Nearest Neighbors": KNeighborsRegressor(),
    "                        Neural Network": MLPRegressor(),
    "Support Vector Machine (Linear Kernel)": LinearSVR(),
    "   Support Vector Machine (RBF Kernel)": SVR(),
    "                         Decision Tree": DecisionTreeRegressor(),
    "                         Random Forest": RandomForestRegressor(),
    "                     Gradient Boosting": GradientBoostingRegressor(),
    "                               XGBoost": XGBRegressor(),
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

                     Linear Regression trained.
 Linear Regression (L2 Regularization) trained.
 Linear Regression (L1 Regularization) trained.
                   K-Nearest Neighbors trained.
                        Neural Network trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                         Decision Tree trained.
                         Random Forest trained.
                     Gradient Boosting trained.
                               XGBoost trained.


In [110]:
#EVALUATE MODEL ON R SQUARED - HIGHER IS BETTER

for name, model in models.items():
    print(name + " R^2 Score: {:.5f}".format(model.score(X_test, y_test)))
    

                     Linear Regression R^2 Score: 0.62648
 Linear Regression (L2 Regularization) R^2 Score: 0.62653
 Linear Regression (L1 Regularization) R^2 Score: -0.00156
                   K-Nearest Neighbors R^2 Score: 0.64066
                        Neural Network R^2 Score: 0.70900
Support Vector Machine (Linear Kernel) R^2 Score: 0.62436
   Support Vector Machine (RBF Kernel) R^2 Score: 0.68944
                         Decision Tree R^2 Score: 0.79960
                         Random Forest R^2 Score: 0.88919
                     Gradient Boosting R^2 Score: 0.88726
                               XGBoost R^2 Score: 0.85756


In [111]:
#EVALUATE MODEL ON RMSE - LOWER IS BETTER

for name, model in models.items():
    y_pred = model.predict(X_test)
    print(name + " RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_test ,y_pred))))
    

                     Linear Regression RMSE: 1.29100
 Linear Regression (L2 Regularization) RMSE: 1.29093
 Linear Regression (L1 Regularization) RMSE: 2.11403
                   K-Nearest Neighbors RMSE: 1.26626
                        Neural Network RMSE: 1.13950
Support Vector Machine (Linear Kernel) RMSE: 1.29466
   Support Vector Machine (RBF Kernel) RMSE: 1.17719
                         Decision Tree RMSE: 0.94562
                         Random Forest RMSE: 0.70317
                     Gradient Boosting RMSE: 0.70927
                               XGBoost RMSE: 0.79723


In [106]:
# GENERATE CSV FOR SUBMISSION TO KAGGLE
# DON'T SUBMIT ALL, ONLY THOSE WHICH DID GOOD ON EVALUATION TO CONFIRM

df = preprocess_inputs(test)

Xs = list(df.columns)
Xs.remove('Index')

X_test = df[Xs]

for name, model in models.items():
    y_pred = model.predict(X_test)
    d = pd.DataFrame(y_pred, columns =['avg_price_per_kg'])
    dff = pd.concat([df['Index'], d], axis=1)
    dff = dff.set_index('Index')
    
    match= re.findall('[A-Z]', name)    
    matchno = re.findall('[0-9]+', name)   
    file_name = ''.join(match) + ''.join(matchno)
    
    dff.to_csv(file_name + '.csv')